In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [38]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (336.43 seconds): Collecting URLs... usatoday            

100.00% (48.46 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25229,25170,1439
yahoo,385,385,174
dailymail,3126,3126,143
reuters,1151,1151,134
euronews,1031,1031,119
rt,396,396,76
aljazeera,479,479,75
metro,239,239,73
breitbart,396,396,46
investing.com,917,917,42


,url,pubs,actual_pub
0,https://cnbc.com/2019/10/18/boeing-shares-slid...,"cnbc, realclearpolitics",cnbc
1,https://foxbusiness.com/markets/opioid-settlem...,"fox, realclearpolitics",fox
2,https://foxbusiness.com/media/verizon-looking-...,"fox, realclearpolitics",fox
3,https://foxbusiness.com/money/andy-puzder-here...,"fox, realclearpolitics",fox
4,https://hotair.com/archives/john-s-2/2019/10/1...,"townhall, hotair",hotair
5,https://msnbc.com/mtp-daily/watch/engel-from-s...,"msnbc, nbcnews",msnbc
6,https://msnbc.com/mtp-daily/watch/full-mcgurk-...,"msnbc, nbcnews",msnbc
7,https://msnbc.com/mtp-daily/watch/joe-biden-fa...,"msnbc, nbcnews",msnbc
8,https://msnbc.com/weekends-with-alex-witt/watc...,"msnbc, nbcnews",msnbc
9,https://nationalreview.com/corner/hillary-clin...,"realclearpolitics, nationalreview",nationalreview


# 2. 해당 URL의 뉴스를 다운로드한다

In [ ]:
download_result = crawler.download(); download_result

89.09% (1326.66 seconds): Downloading... dailymail           